In [ ]:
import os
!git clone https://github.com/zhaoyutim/ViirsTimeSeriesModel.git
os.chdir('ViirsTimeSeriesModel')

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

from model.vit.vit_model import VisionTransformerGenerator

if __name__ == '__main__':
    MAX_EPOCHS = 100
    dataset = np.load('../data/proj3_training.npy').transpose((1,0,2))
    print(dataset.shape)

    positive_sample = dataset[(dataset[:,:,45]>0).any(axis=1)]
    negative_sample = dataset[(dataset[:,:,45]==0).any(axis=1)]
    negative_sample = negative_sample[np.random.choice(negative_sample.shape[0], positive_sample.shape[0])]
    dataset = np.concatenate((positive_sample,negative_sample), axis=0)
    y_dataset = np.zeros((dataset.shape[0],dataset.shape[1],2))
    y_dataset[: ,:, 0] = dataset[:, :, 45] > 0
    y_dataset[:, :, 1] = dataset[:, :, 45] == 0

    x_train, x_test, y_train, y_test = train_test_split(dataset[:,:,:45], y_dataset, test_size=0.2)
    input_shape = x_train[0,:,:].shape
    num_classes=2
    projection_dim = 128
    num_heads = 4
    transformer_units = [
        projection_dim * 2,
        projection_dim,
    ]  # Size of the transformer layers
    transformer_layers = 8

    mlp_head_units = [2048, 1024]

    vit_gen = VisionTransformerGenerator(input_shape, projection_dim, transformer_layers, num_heads, mlp_head_units, num_classes)

    history = vit_gen.run_experiment(x_train, y_train, x_test, y_test, batch_size=256, num_epochs=100, learning_rate=0.001, weight_decay=0.0001)